# Project 4 Bean Leaves ANN

Ashton Cole

AVC697

COE 379L: Software Design for Responsive Intelligent Systems

## Description

This notebook performs a hyperparameter sweep of a densely-connected Artificial Neural Network applied to a bean leaf data set. Images of bean leaves are classified as either healthy, having rust disease, or angular leaf spot disease.

*Note: This notebook may take a long time to run. It trains and tests 16 dense neural networks.*

In [1]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

2024-05-02 23:12:36.212626: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 23:12:36.251333: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 23:12:36.251376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 23:12:36.253062: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-02 23:12:36.260438: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 23:12:36.260935: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Preprocessing

In [2]:
validation_split = 0.2 # Validation split
batch_size = 32
seed = 123

image_s = 28

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

## Model Construction and Compilation

In [3]:
image_length = image_s * image_s

optimizer='adam'
loss='sparse_categorical_crossentropy'
metrics=['accuracy']

def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(image_s, image_s))) # F1
    model.add(Dense(hp.Choice('first', [64, 128, 192, 256]), activation='relu')) # F2
    model.add(Dense(hp.Choice('second', [64, 128, 192, 256]), activation='relu')) # F3
    model.add(Dense(10, activation='softmax')) # Output
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )
    return model

## Model Parameter Search

In [4]:
directory = 'tuner'
project_name = 'ann-mnist'

tuner = kt.GridSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    overwrite=False,
    directory=directory,
    project_name=project_name
)

tuner.search_space_summary()

Search space summary
Default search space size: 2
first (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 192, 256], 'ordered': True}
second (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 192, 256], 'ordered': True}


In [5]:
logger = CSVLogger(f'{project_name}.csv', append=True)

tuner.search(
    x=X_train_scaled,
    y=y_train,
    epochs=10,
    validation_split=validation_split,
    verbose=2,
    batch_size=batch_size,
    callbacks=[logger]
)

Trial 16 Complete [00h 01m 00s]
val_accuracy: 0.9779999852180481

Best val_accuracy So Far: 0.9788333177566528
Total elapsed time: 00h 12m 07s


## Model Testing

In [6]:
trials = tuner.oracle.trials

for (id, trial) in trials.items():
    print(f'Trial {id}')
    print(trial.summary())
    model = tuner.load_model(trial)
    test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
    print(f'Test accuracy: {test_accuracy}')
    print('--------------------')

Trial 0000
Trial 0000 summary
Hyperparameters:
first: 64
second: 64
Score: 0.9746666550636292
None
Test accuracy: 0.9728999733924866
--------------------
Trial 0001
Trial 0001 summary
Hyperparameters:
first: 64
second: 128
Score: 0.9757500290870667
None
Test accuracy: 0.9749000072479248
--------------------
Trial 0002
Trial 0002 summary
Hyperparameters:
first: 64
second: 192
Score: 0.9738333225250244
None
Test accuracy: 0.973800003528595
--------------------
Trial 0003
Trial 0003 summary
Hyperparameters:
first: 64
second: 256
Score: 0.9734166860580444
None
Test accuracy: 0.9726999998092651
--------------------
Trial 0004
Trial 0004 summary
Hyperparameters:
first: 128
second: 64
Score: 0.9760000109672546
None
Test accuracy: 0.9768999814987183
--------------------
Trial 0005
Trial 0005 summary
Hyperparameters:
first: 128
second: 128
Score: 0.9745833277702332
None
Test accuracy: 0.9763000011444092
--------------------
Trial 0006
Trial 0006 summary
Hyperparameters:
first: 128
second: 192
S